# 🧪 Consigna de Evaluación Parcial - Procesamiento de Texto y Clustering

## 🎯 Objetivo
#### Aplicar técnicas de procesamiento de texto y representación vectorial para agrupar comentarios en español en función de su contenido semántico, utilizando técnicas de clustering no supervisado.

In [1]:
# Importaciones necesarias
import pandas as pd
import nltk
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Procesamiento
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lolonastri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lolonastri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lolonastri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Lista de stopwords en inglés
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [4]:
# Cargar el dataset IMDb (en inglés)
dataset = load_dataset("imdb")

In [5]:
# Tomamos una muestra de 1000 textos
df = pd.DataFrame(dataset['train']).sample(frac=0.02, random_state=42).reset_index(drop=True)

In [6]:
# Mostramos los primeros textos originales
print("Ejemplos originales:")
for i in range(3):
    print(f"{i+1}. {df['text'][i][:200]}...\n")

Ejemplos originales:
1. Dumb is as dumb does, in this thoroughly uninteresting, supposed black comedy. Essentially what starts out as Chris Klein trying to maintain a low profile, eventually morphs into an uninspired version...

2. I dug out from my garage some old musicals and this is another one of my favorites. It was written by Jay Alan Lerner and directed by Vincent Minelli. It won two Academy Awards for Best Picture of 195...

3. After watching this movie I was honestly disappointed - not because of the actors, story or directing - I was disappointed by this film advertisements.<br /><br />The trailers were suggesting that the...



---

## 1. Preprocesamiento de texto

Aplicá las siguientes tareas sobre una columna de comentarios o textos en español:

- **Tokenización:** Convertí cada texto en una secuencia de palabras.
- **Eliminación de stopwords:** Remové palabras vacías utilizando una lista en español.
- **Stemming o Lematización:** Elegí una de las dos técnicas y aplicala sobre los tokens procesados.

📌 *Mostrá ejemplos del texto antes y después del preprocesamiento.*

In [7]:
import string
# Función de preprocesamiento
def preprocess_text(text):
    tokens = text.lower().split()  # tokenización básica
    tokens = [t.strip(string.punctuation) for t in tokens if t.isalpha()]  # limpiar signos
    tokens = [t for t in tokens if t not in stop_words]  # quitar stopwords
    tokens = [lemmatizer.lemmatize(t) for t in tokens]  # lematizar
    return tokens


In [8]:
# Columna con tokens preprocesados
df['tokens'] = df['text'].apply(preprocess_text)

# Mostrar ejemplos antes y después
for i in range(3):
    print(f"\n📄 Texto original #{i+1}:\n{df['text'][i][:300]}...\n")
    print(f"✅ Tokens procesados #{i+1}:\n{df['tokens'][i][:30]}...\n")


📄 Texto original #1:
Dumb is as dumb does, in this thoroughly uninteresting, supposed black comedy. Essentially what starts out as Chris Klein trying to maintain a low profile, eventually morphs into an uninspired version of "The Three Amigos", only without any laughs. In order for black comedy to work, it must be outra...

✅ Tokens procesados #1:
['dumb', 'dumb', 'thoroughly', 'supposed', 'black', 'essentially', 'start', 'chris', 'klein', 'trying', 'maintain', 'low', 'eventually', 'morphs', 'uninspired', 'version', 'three', 'without', 'order', 'black', 'comedy', 'must', 'order', 'black', 'comedy', 'cannot', 'mean', 'really', 'town', 'full']...


📄 Texto original #2:
I dug out from my garage some old musicals and this is another one of my favorites. It was written by Jay Alan Lerner and directed by Vincent Minelli. It won two Academy Awards for Best Picture of 1951 and Best Screenplay. The story of an American painter in Paris who tries to make it big. Nina Foch...

✅ Tokens procesado

---

## 2. Representación vectorial

Convertí los textos preprocesados en vectores numéricos utilizando alguna de las siguientes técnicas:

- Bag-of-Words (BoW)
- TF-IDF (preferido si vas a comparar pesos entre palabras)

📌 *Visualizá brevemente la matriz resultante y contá cuántas dimensiones tiene.*

# 1) BoW

In [9]:
# Unir los tokens preprocesados en un string nuevamente
df['clean_text'] = df['tokens'].apply(lambda tokens: ' '.join(tokens))

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# Crear vectorizador BoW
bow_vectorizer = CountVectorizer()
X_bow = bow_vectorizer.fit_transform(df['clean_text'])

print(f"🧱 Matriz BoW: {X_bow.shape}")
print(f"📄 Ejemplo de palabras:\n{bow_vectorizer.get_feature_names_out()[:20]}")

🧱 Matriz BoW: (500, 8993)
📄 Ejemplo de palabras:
['aamir' 'abandon' 'abandoned' 'abashed' 'abby' 'abc' 'abdominal'
 'abducted' 'abducting' 'abetted' 'ability' 'abject' 'able' 'aboard'
 'abominable' 'abomination' 'aboriginal' 'aborted' 'abound' 'abroad']


# 2) TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Transformar el texto a vectores numéricos
X_tfidf = tfidf_vectorizer.fit_transform(df['clean_text'])

In [12]:
# Mostrar forma de la matriz y algunas palabras
print(f"✅ La matriz TF-IDF tiene forma: {X_tfidf.shape}")
print(f"🧠 Número de dimensiones (palabras únicas): {len(tfidf_vectorizer.get_feature_names_out())}")

✅ La matriz TF-IDF tiene forma: (500, 8993)
🧠 Número de dimensiones (palabras únicas): 8993


In [13]:
# Ver algunas features (palabras del vocabulario)
print(f"\n🔤 Algunas palabras del vocabulario:\n{tfidf_vectorizer.get_feature_names_out()[:20]}")


🔤 Algunas palabras del vocabulario:
['aamir' 'abandon' 'abandoned' 'abashed' 'abby' 'abc' 'abdominal'
 'abducted' 'abducting' 'abetted' 'ability' 'abject' 'able' 'aboard'
 'abominable' 'abomination' 'aboriginal' 'aborted' 'abound' 'abroad']


# 3) Embeddings

---

## 3. Agrupamiento (Clustering)

Aplicá un algoritmo de clustering no supervisado para agrupar los comentarios:

- Puede ser **KMeans**, **Agglomerative Clustering** o **DBSCAN**.
- Determiná el número de clusters si es necesario (por ejemplo, usando el método del codo para KMeans).
- Asigná cada comentario a un grupo (cluster).

📌 *Visualizá los clusters utilizando reducción de dimensionalidad (por ejemplo con PCA, t-SNE o UMAP).*

---

## 4. Análisis e interpretación

- Describí brevemente qué patrones observás en cada cluster.
- Mostrá ejemplos representativos de cada grupo.
- Discutí si los clusters parecen alinearse con diferentes tonos, temas o polaridades de los textos.


---

## 💡 Bonus (opcional)

- Etiquetá manualmente una muestra de los textos con polaridad (positivo/negativo) y analizá si los clusters coinciden con estas etiquetas.
- Compará los resultados usando TF-IDF vs BoW.